In [ ]:
from torch import cuda

print(cuda.is_available())
print(cuda.get_device_name())

In [ ]:
import torch

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

import matplotlib as plt

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
batch_size = 64

# Create dataloaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)

In [ ]:
# Create model.
device = "cuda" if cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

In [ ]:
# A model is trained using certain data provided by dataloader
# to optimize itself based on certain loss function loss_fn and
# an optimizer on certain device ("cuda" or "cpu").

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    model.train() # Set model on training mode.
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute error.
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training progress.
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

# Test model using certain test data provided by dataloader using
# loss function loss_fn on certain device ("cuda" or "cpu").
def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # Set model on evaluation mode.
    test_loss, correct = 0, 0
    with torch.no_grad(): # Eval model by running model without gradient descent.
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test error: \n Accuracy: {(100*correct):>0.1f}% \n Avg Loss: {test_loss:>8f} \n")


In [ ]:
epochs = 5
for e in range(epochs):
    print(f"Training on epoch {e+1} \n")
    train(train_dataloader, model, loss_fn, optimizer, device)
    test(test_dataloader, model, loss_fn, device)
print("done!")

In [ ]:
# Saving model.
import os
if not os.path.exists('./model'):
    os.mkdir('model')
torch.save(model.state_dict(), "model/tutorial_model.pth")

In [ ]:
# Load model.
m = NeuralNetwork()
m.load_state_dict(torch.load("./model/tutorial_model.pth"))


In [ ]:
# Use loaded model for prediction.
labels = [
    "tshirt", "trousers", "pullover", "dress", "coat", "sandal", "shirt", "sneaker", "bag", "ankle_boot"
]

m.eval() # Set model on evaluation mode.
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = m(x)
    predicted, actual = labels[pred[0].argmax(0)], labels[y]
    print("prediction: {}, actual: {}".format(predicted, actual))

pred
test_data[0][1]